# Convolutional Neural Network

### Importing the libraries

In [80]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [81]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [82]:
train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True
                                  rotation_range=20,           # Rotate the image by up to 20 degrees
    width_shift_range=0.2,       # Shift the image horizontally by up to 20% of the image width
    height_shift_range=0.2,      # Shift the image vertically by up to 20% of the image height
    shear_range=0.2,             # Shear the image by up to 20 degrees
    zoom_range=0.2,              # Zoom in or out on the image by up to 20%
    horizontal_flip=True,        # Flip the image horizontally
    fill_mode='nearest'
                                  
                                  )


training_set = train_datagen.flow_from_directory('trainset\yellowing_of_skin',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 12 images belonging to 2 classes.


### Preprocessing the Test set

In [83]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('testset\yellowing_of_skin',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [84]:
cnn = tf.keras.models.Sequential()



### Step 1 - Convolution

In [85]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [86]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [87]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [88]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [89]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [90]:
cnn.add(tf.keras.layers.Dense(units=2, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [91]:
cnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', 
              metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [92]:
import os

dir_path = r'C:\Users\Geetha\Desktop\std_severity\trainset\yellowing_of_skin'

# Get a list of all the files in the directory
file_list = os.listdir(dir_path)

# Loop through the files and delete any with the ".ini" extension
for file_name in file_list:
    if file_name.endswith('.ini'):
        file_path = os.path.join(dir_path, file_name)
        os.remove(file_path)
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\muscle_wasting\desktop.ini')
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\yellowing_skin\desktop.ini')


In [93]:
import os
import cv2
bad_list=[]
dir=r'C:\Users\Geetha\Desktop\std_severity\trainset\yellowing_of_skin'
subdir_list=os.listdir(dir)
for d in subdir_list:  
    dpath=os.path.join (dir, d) 
    if d in ['testset', 'trainset']:
        class_list=os.listdir(dpath) 
       # print (class_list)
        for klass in class_list: 
            class_path=os.path.join(dpath, klass) 
            #print(class_path)
            file_list=os.listdir(class_path) 
            for f in file_list: 
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') 
                ext=f[index+1:] 
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)

[]


In [94]:
cnn.fit(x = training_set, validation_data=test_set , epochs = 25)


Epoch 1/25
1/1 [==============================] - 2s 2s/step - loss: 0.7147 - accuracy: 0.2500 - val_loss: 1.7061 - val_accuracy: 0.5000
Epoch 2/25
1/1 [==============================] - 0s 248ms/step - loss: 1.1079 - accuracy: 0.7500 - val_loss: 0.9118 - val_accuracy: 0.5000
Epoch 3/25
1/1 [==============================] - 0s 267ms/step - loss: 0.6538 - accuracy: 0.7500 - val_loss: 0.5868 - val_accuracy: 0.5000
Epoch 4/25
1/1 [==============================] - 0s 239ms/step - loss: 0.5629 - accuracy: 0.8333 - val_loss: 0.5095 - val_accuracy: 1.0000
Epoch 5/25
1/1 [==============================] - 0s 269ms/step - loss: 0.5232 - accuracy: 1.0000 - val_loss: 0.5175 - val_accuracy: 0.5000
Epoch 6/25
1/1 [==============================] - 0s 234ms/step - loss: 0.4312 - accuracy: 0.7500 - val_loss: 0.6799 - val_accuracy: 0.5000
Epoch 7/25
1/1 [==============================] - 0s 266ms/step - loss: 0.4162 - accuracy: 0.7500 - val_loss: 0.5640 - val_accuracy: 0.5000
Epoch 8/25
1/1 [=======

In [95]:

class_indices = training_set.class_indices
print(class_indices)

# Get the class label for a particular class




{'normal skin': 0, 'turned yellow': 1}


## Part 4 - Making a single prediction

In [97]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('single_prediction/yellow_skin.jpg', target_size = (64, 64))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)
training_set.class_indices
if result[0][0] >result[0][1] :
  prediction = 'normal skin'
elif result[0][1] >result[0][0] :
  prediction = 'turned yellow'



probs = cnn.predict(test_image)
print(probs)

prob_sum = np.sum(probs)
print(prob_sum)

indices=np.argsort(probs)
print(indices)
print(probs[0][indices])
if probs[0][indices[0][0]]> probs[0][indices[0][1]]:
    print("a is larger than b")
else:
    print("b is larger than a")
    
# top_3 = np.argsort(probs,axis=1)[:, -3:]
# print(top_3)
# class_probs = np.take_along_axis(probs, top_3, axis=1)
# print(class_probs)

# print(f"The test sample {test_image} ")
# print("Top three Predicted classes and their corresponding probabilties\n")
# for j in range(3):
#         class_idx = top_3[0][2-j]
#         class_prob = class_probs[0][2-j]
#         print(f"\tClass {class_idx}: {class_prob:.5f}")
# print(f"Actual class to which the test sample belongs to {}  ")�


1/1 [==============================] - 0s 28ms/step
[[0. 1.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1.]]
1.0
[[0 1]]
[[0. 1.]]
b is larger than a
[[0 1]]
[[0. 1.]]
The test sample [[[[255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]
   ...
   [255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]]

  [[255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]
   ...
   [255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]]

  [[255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]
   ...
   [255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]]

  ...

  [[255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]
   ...
   [255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]]

  [[255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]
   ...
   [255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]]

  [[255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]
   ...
   [255. 255. 255.]
   [255. 255. 255.]
   [255. 255. 255.]]]] 
Top three Predicte

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
print(prediction)

In [ ]:
print(result)